In [ ]:
import asyncio
import time

class ThrottledQueue(asyncio.Queue):
    "subclass asyncio.Queue i.e. import all behaviour"

    def __init__(self, per_second, maxsize=0, *, loop=None, i=0):
        "Set up some extra vars and then call the original init"

        self.lock = asyncio.Lock()
        self.i = i
        self.per_second = per_second
        self.last_get = time.time() # this is the fastest way... I think?
        super(ThrottledQueue, self).__init__(maxsize=maxsize, loop=loop)

    async def retry(self):
        """
        Signals to the queue that an item is being retried, 
        so pause any get()s by aquiring the lock and throttling before releasing
        """
        async with self.lock:
            await self._throttle()

    async def get(self):
        async with self.lock:
            await self._throttle()
            result = await super(ThrottledQueue, self).get()

            self.last_get = time.time()
            return result

    async def _throttle(self):
        elapsed = time.time() - self.last_get
        sleep_time = self.per_second - elapsed
        print(self.i, '- times', f'{elapsed:.5f}', f'{sleep_time:.5f}', '- sizes', self.qsize(), f'{self.qsize() / self.maxsize:.5f}')
        await asyncio.sleep(max(0, sleep_time)) # Make sure we wait at least 0 seconds
